# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt

from torch import cuda

from absa_models.ABSA_BERT_Dropout_CNN_BiLSTM_Linear import ABSA_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error()

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8
SEQ_LEN = 512

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
MODEL_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_cnn_bilstm_linear_512.pth'
STATS_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_cnn_bilstm_linear_512.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    losses = []

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))

        losses.append(loss.item())
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()
    
    return losses

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_CNN_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    train_losses = []

    for epoch in range(EPOCHS):
        losses = train(epoch, model, loss_fn, optimizer, train_dataloader)

        train_losses += losses

    plt.title(f'Train Loss for run {i + 1}/{NO_RUNS}')
    plt.plot(train_losses)
    plt.savefig(f'../../../results/ABSA/SemEval16 - Task 5 - Restaurants/plots/bert_pt_do_cnn_bilstm_lin/train_loss_run_{i + 1}.png')

    plt.clf()

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4244142770767212
Epoch: 0/2, Batch: 50/501, Loss: 0.7621605396270752
Epoch: 0/2, Batch: 100/501, Loss: 0.16898216307163239
Epoch: 0/2, Batch: 150/501, Loss: 0.07719364762306213
Epoch: 0/2, Batch: 200/501, Loss: 0.052880626171827316
Epoch: 0/2, Batch: 250/501, Loss: 0.05863593518733978
Epoch: 0/2, Batch: 300/501, Loss: 0.04053518921136856
Epoch: 0/2, Batch: 350/501, Loss: 0.053899310529232025
Epoch: 0/2, Batch: 400/501, Loss: 0.02590976655483246
Epoch: 0/2, Batch: 450/501, Loss: 0.033876173198223114
Epoch: 0/2, Batch: 500/501, Loss: 0.03617732599377632
Epoch: 1/2, Batch: 0/501, Loss: 0.020049378275871277
Epoch: 1/2, Batch: 50/501, Loss: 0.031808916479349136
Epoch: 1/2, Batch: 100/501, Loss: 0.037650253623723984
Epoch: 1/2, Batch: 150/501, Loss: 0.02042081020772457
Epoch: 1/2, Batch: 200/501, Loss: 0.023631004616618156
Epoch: 1/2, Batch: 250/501, Loss: 0.02649899572134018
Epoch: 1/2, Batch: 300/501, Loss: 0.009462804533541203
Epoch: 1/2, Batch: 

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3823485374450684
Epoch: 0/2, Batch: 50/501, Loss: 0.7779449224472046
Epoch: 0/2, Batch: 100/501, Loss: 0.20638726651668549
Epoch: 0/2, Batch: 150/501, Loss: 0.11184384673833847
Epoch: 0/2, Batch: 200/501, Loss: 0.054087940603494644
Epoch: 0/2, Batch: 250/501, Loss: 0.056266989558935165
Epoch: 0/2, Batch: 300/501, Loss: 0.05231497064232826
Epoch: 0/2, Batch: 350/501, Loss: 0.03632121533155441
Epoch: 0/2, Batch: 400/501, Loss: 0.03279614448547363
Epoch: 0/2, Batch: 450/501, Loss: 0.03905732184648514
Epoch: 0/2, Batch: 500/501, Loss: 0.02300729788839817
Epoch: 1/2, Batch: 0/501, Loss: 0.017945103347301483
Epoch: 1/2, Batch: 50/501, Loss: 0.02195914275944233
Epoch: 1/2, Batch: 100/501, Loss: 0.04766159504652023
Epoch: 1/2, Batch: 150/501, Loss: 0.025572732090950012
Epoch: 1/2, Batch: 200/501, Loss: 0.02589157596230507
Epoch: 1/2, Batch: 250/501, Loss: 0.031018631532788277
Epoch: 1/2, Batch: 300/501, Loss: 0.023107057437300682
Epoch: 1/2, Batch: 35

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4022974967956543
Epoch: 0/2, Batch: 50/501, Loss: 0.5822946429252625
Epoch: 0/2, Batch: 100/501, Loss: 0.18358011543750763
Epoch: 0/2, Batch: 150/501, Loss: 0.08579973876476288
Epoch: 0/2, Batch: 200/501, Loss: 0.05762350931763649
Epoch: 0/2, Batch: 250/501, Loss: 0.05097316950559616
Epoch: 0/2, Batch: 300/501, Loss: 0.06596489995718002
Epoch: 0/2, Batch: 350/501, Loss: 0.03518326207995415
Epoch: 0/2, Batch: 400/501, Loss: 0.04147946089506149
Epoch: 0/2, Batch: 450/501, Loss: 0.04013121873140335
Epoch: 0/2, Batch: 500/501, Loss: 0.023095794022083282
Epoch: 1/2, Batch: 0/501, Loss: 0.02464231662452221
Epoch: 1/2, Batch: 50/501, Loss: 0.0189609844237566
Epoch: 1/2, Batch: 100/501, Loss: 0.029560208320617676
Epoch: 1/2, Batch: 150/501, Loss: 0.021789034828543663
Epoch: 1/2, Batch: 200/501, Loss: 0.0175994411110878
Epoch: 1/2, Batch: 250/501, Loss: 0.012159929610788822
Epoch: 1/2, Batch: 300/501, Loss: 0.018855733796954155
Epoch: 1/2, Batch: 350/5

...

Run 9/10
Epoch: 0/2, Batch: 0/501, Loss: 1.385345220565796
Epoch: 0/2, Batch: 50/501, Loss: 0.5833980441093445
Epoch: 0/2, Batch: 100/501, Loss: 0.1702398955821991
Epoch: 0/2, Batch: 150/501, Loss: 0.09140435606241226
Epoch: 0/2, Batch: 200/501, Loss: 0.06694945693016052
Epoch: 0/2, Batch: 250/501, Loss: 0.035450126975774765
Epoch: 0/2, Batch: 300/501, Loss: 0.035219546407461166
Epoch: 0/2, Batch: 350/501, Loss: 0.035807736217975616
Epoch: 0/2, Batch: 400/501, Loss: 0.02107243798673153
Epoch: 0/2, Batch: 450/501, Loss: 0.020557578653097153
Epoch: 0/2, Batch: 500/501, Loss: 0.020545698702335358
Epoch: 1/2, Batch: 0/501, Loss: 0.034113410860300064
Epoch: 1/2, Batch: 50/501, Loss: 0.024700820446014404
Epoch: 1/2, Batch: 100/501, Loss: 0.026117250323295593
Epoch: 1/2, Batch: 150/501, Loss: 0.03648308664560318
Epoch: 1/2, Batch: 200/501, Loss: 0.01687144674360752
Epoch: 1/2, Batch: 250/501, Loss: 0.032875917851924896
Epoch: 1/2, Batch: 300/501, Loss: 0.023400476202368736
Epoch: 1/2, Batch: 

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/501, Loss: 1.40829598903656
Epoch: 0/2, Batch: 50/501, Loss: 0.95563805103302
Epoch: 0/2, Batch: 100/501, Loss: 0.24524208903312683
Epoch: 0/2, Batch: 150/501, Loss: 0.12684805691242218
Epoch: 0/2, Batch: 200/501, Loss: 0.05401114374399185
Epoch: 0/2, Batch: 250/501, Loss: 0.04055645316839218
Epoch: 0/2, Batch: 300/501, Loss: 0.04396292567253113
Epoch: 0/2, Batch: 350/501, Loss: 0.023701680824160576
Epoch: 0/2, Batch: 400/501, Loss: 0.03166460990905762
Epoch: 0/2, Batch: 450/501, Loss: 0.024580134078860283
Epoch: 0/2, Batch: 500/501, Loss: 0.02612486109137535
Epoch: 1/2, Batch: 0/501, Loss: 0.026406394317746162
Epoch: 1/2, Batch: 50/501, Loss: 0.07605068385601044
Epoch: 1/2, Batch: 100/501, Loss: 0.0286334790289402
Epoch: 1/2, Batch: 150/501, Loss: 0.03778694570064545
Epoch: 1/2, Batch: 200/501, Loss: 0.018423005938529968
Epoch: 1/2, Batch: 250/501, Loss: 0.028700698167085648
Epoch: 1/2, Batch: 300/501, Loss: 0.030106758698821068
Epoch: 1/2, Batch: 350/50

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<Figure size 432x288 with 0 Axes>

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.994953,0.994953,0.404311,0.994953,0.252189,0.994953,0.253635,343.167196
1,0.995020,0.995020,0.248760,0.995020,0.250000,0.995020,0.249378,340.724913
2,0.994945,0.994945,0.512644,0.994945,0.251603,0.994945,0.252414,339.733481
3,0.994871,0.994871,0.498740,0.994871,0.250244,0.994871,0.249864,344.962219
4,0.994805,0.994805,0.436763,0.994805,0.258551,0.994805,0.265569,372.016572
5,0.995234,0.995234,0.248809,0.995234,0.250000,0.995234,0.249403,380.239597
6,0.994754,0.994754,0.373689,0.994754,0.250251,0.994754,0.249846,371.781314
7,0.994953,0.994953,0.248740,0.994953,0.250000,0.994953,0.249369,361.676120
8,0.994949,0.994949,0.373745,0.994949,0.250514,0.994949,0.250398,388.845268
9,0.994723,0.994723,0.248681,0.994723,0.250000,0.994723,0.249339,385.609190


In [20]:
results.to_csv(STATS_OUTPUT)